In [1]:
###############
### Imports ###
###############

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
#########################
### Gather Data Files ###
#########################


req = requests.get('https://vehiculeselectriques.gouv.qc.ca/english/rabais/ve-neuf/vehicules-neufs-admissibles.asp')
soup = BeautifulSoup(req.content, 'html.parser')
with open('Eligible vehicles.html') as Eliveh:
    new_soup = BeautifulSoup(Eliveh,'html.parser')
cars = soup.find_all("div", class_="infoVoiture")
fed_cars = new_soup.find('table')
fed_cars = fed_cars.findAll('tr', role='row')[1:]

In [3]:
###########################
### Load into Dataframe ###
###########################


fed_cars_df = pd.DataFrame()

for car in fed_cars:
    car_make = car.find('td', class_='sorting_1').text
    car_model = car.find('td', class_='sorting_3').text
    car_model_year = car.find('td', class_='sorting_2').text
    try:
        car_trim = car.find('td', headers='tbl8').text
        car_category = car.find('td', headers='tbl9').text
        car_rebate = car.find('td', headers='tbl12').text
        car_eligibility = car.find('td', headers = 'tbl16').text
    except:
        car_trim = ''
        car_category = ''
        car_rebate = ''
        car_eligibility = 'Empty'
    fed_cars_df = fed_cars_df.append({'Make':car_make,'Model':car_model, 'Trim':car_trim,'Rebate-Fed':car_rebate,'Year-Fed':car_model_year,'Category':car_category,'Eligibility':car_eligibility}, ignore_index=True)

cars_df = pd.DataFrame()
makes = ['Audi','BMW','BYD','Chevrolet','Chrysler','Ford','Honda','Hyundai','Jeep','Kia','Lexus','Lincoln','Mazda','Mini','Mitsubishi','Nissan','Polestar','Subaru','Tesla','Toyota','Volkswagen','Volvo']

for car in cars:
    car_category = car.findAll('p')[1].text[10:]
    if car_category in ['All-electric','Plug-in hybrid']:
        car_string = car.find('h2').text
        car_model_year = car.findAll('p')[0].text[15:-2]
        car_rebate = car.find('div', class_='rabais-achat').text[1:-1]
        for make in makes:
            if make in car_string:
                car_name = car_string[:len(make)]
                car_model = car_string[len(make)+1:]
                continue
        cars_df = cars_df.append({'Make':car_name,'Model':car_model,'Rebate-QC':car_rebate,'Year-QC':car_model_year,'Category':car_category}, ignore_index=True)

fed_cars_df.loc[fed_cars_df['Eligibility'] == 'Empty']

,Make,Model,Trim,Rebate-Fed,Year-Fed,Category,Eligibility
12,BMW,i3,,,2020,,Empty
27,Ford,*Escape PHEV,,,2022,,Empty
30,Ford,*Escape PHEV,,,2021,,Empty
32,Ford,*Escape PHEV,,,2020,,Empty
41,Hyundai,*Santa Fe PHEV,,,2022,,Empty
42,Hyundai,*Tucson PHEV,,,2022,,Empty
63,Kia,*Sorento PHEV,,,2022,,Empty


This is all the values that did not enter correctly from the web scraping

In [4]:
#####################################
### Manually Enter Missing Values ###
#####################################

veh_list = [12,27,30,32,41,42,63]

rebate_qc = ['$5000','$5000','$5000','$5000','$5000','$5000','$5000']
category = ['BEV','PHEV','PHEV','PHEV','PHEV','PHEV','PHEV']
trim = ['Base\n\t\t\ts','SE\n\t\t\tSEL\n\t\t\tTitanium','SE\n\t\t\tSEL\n\t\t\tTitanium','SE\n\t\t\tSEL\n\t\t\tTitanium','Preferred\n\t\t\tLuxury','Preferred\n\t\t\tLuxury','EX\n\t\t\tEX+\n\t\t\tSX']
eligibility = ['2020-01-16','2022-04-25','2022-04-25','2022-04-25','2022-04-25','2022-04-25','2022-04-25']

for ind, car in enumerate(veh_list):
    fed_cars_df.iloc[car,2] = trim[ind]
    fed_cars_df.iloc[car,3] = rebate_qc[ind]
    fed_cars_df.iloc[car,-2] = category[ind]
    fed_cars_df.iloc[car,-1] = eligibility[ind]

fed_cars_df.iloc[veh_list,:]

,Make,Model,Trim,Rebate-Fed,Year-Fed,Category,Eligibility
12,BMW,i3,Base\n\t\t\ts,$5000,2020,BEV,2020-01-16
27,Ford,*Escape PHEV,SE\n\t\t\tSEL\n\t\t\tTitanium,$5000,2022,PHEV,2022-04-25
30,Ford,*Escape PHEV,SE\n\t\t\tSEL\n\t\t\tTitanium,$5000,2021,PHEV,2022-04-25
32,Ford,*Escape PHEV,SE\n\t\t\tSEL\n\t\t\tTitanium,$5000,2020,PHEV,2022-04-25
41,Hyundai,*Santa Fe PHEV,Preferred\n\t\t\tLuxury,$5000,2022,PHEV,2022-04-25
42,Hyundai,*Tucson PHEV,Preferred\n\t\t\tLuxury,$5000,2022,PHEV,2022-04-25
63,Kia,*Sorento PHEV,EX\n\t\t\tEX+\n\t\t\tSX,$5000,2022,PHEV,2022-04-25


In [5]:
##################
###save to file###
##################

fed_cars_df.to_csv('fed_cars_df_master.csv')
cars_df.to_csv('cars_df_master.csv')

In [40]:
######################
### Load from file ###
######################

fed_cars_df = pd.read_csv('fed_cars_df.csv',index_col=0)
cars_df = pd.read_csv('cars_df_master.csv',index_col= 0)

In [6]:
def trimseparator(series,separator = '\n\t\t\t|\n'):
    str = series['Trim']
    res = re.split(separator,str)
    for ind in res:
        car_make = series['Make']
        car_model = series['Model']
        car_model_year = series['Year-Fed']
        car_category = series['Category']
        car_rebate = series['Rebate-Fed']
        car_eligibility = series['Eligibility']
        temp_dict={'Make':car_make,'Model':car_model, 'Trim':ind,'Rebate-Fed':car_rebate,'Year-Fed':car_model_year,'Category':car_category,'Eligibility':car_eligibility}
    return pd.DataFrame(temp_dict)

def timeseparator(series,separator = '\n\t\t\t'):
    str = series['Year-Fed']
    res = str.split(separator)
    for ind in res:
        car_make = series['Make']
        car_model = series['Model']
        car_trim = series['Trim']
        car_category = series['Category']
        car_rebate = series['Rebate-Fed']
        car_eligibility = series['Eligibility']
        temp_dict={'Make':car_make,'Model':car_model, 'Trim':car_trim,'Rebate-Fed':car_rebate,'Year-Fed':ind,'Category':car_category,'Eligibility':car_eligibility}
    return pd.DataFrame(temp_dict)

size = range(len(fed_cars_df))



drop_list = []
separators = ['\n\t\t\t', '\n']
for ind in size:
    series = fed_cars_df.iloc[ind]
    if any([separator in series['Trim'] for separator in separators]):
        temp= trimseparator(series)
        fed_cars_df = fed_cars_df.append(temp, ignore_index=True)
        drop_list.append(ind)
fed_cars_df.drop(drop_list,inplace=True)

size = range(len(fed_cars_df))

for ind in size:
    series = fed_cars_df.iloc[ind]
    if '/' in series['Year-Fed']:
        if '/\n\t\t\t' in series['Year-Fed']:
            series['Year-Fed'] = series['Year-Fed'].replace('/\n\t\t\t', '\n\t\t\t')
        else:
            series['Year-Fed'] = series['Year-Fed'].replace('/', '\n\t\t\t')

drop_list = []
separators = ['\n\t\t\t', '\n']
for ind in size:
    series = fed_cars_df.iloc[ind]
    if any([separator in series['Year-Fed'] for separator in separators]):
        temp= timeseparator(series)
        fed_cars_df = fed_cars_df.append(temp, ignore_index=True)
        drop_list.append(ind)
fed_cars_df.drop(drop_list,inplace=True)

The database is now separated by trim and year released.

In [22]:
#########################
### Manual Processing ###
#########################

temp_dict = {'Make': [cars_df.iloc[4,0],cars_df.iloc[4,0]],'Model':['i3 – REx','i3s – Rex'],'Rebate-QC':[cars_df.iloc[4,2],cars_df.iloc[4,2]],'Year-QC':[cars_df.iloc[4,3],cars_df.iloc[4,3]],'Category':[cars_df.iloc[4,4],cars_df.iloc[4,4]]}
temp_dict2 = {'Make': [cars_df.iloc[38,0],cars_df.iloc[38,0]],'Model':['Leaf','Leaf Plus'],'Rebate-QC':[cars_df.iloc[38,2],cars_df.iloc[38,2]],'Year-QC':[cars_df.iloc[38,3],cars_df.iloc[38,3]],'Category':[cars_df.iloc[38,4],cars_df.iloc[38,4]]}

i3_df = pd.DataFrame(temp_dict)
leaf_df = pd.DataFrame(temp_dict2)

cars_df = cars_df.append(i3_df,ignore_index=True)
cars_df = cars_df.append(leaf_df,ignore_index=True)
cars_df.drop(4, inplace=True)
cars_df.drop(38, inplace=True)

##########################
### General Processing ###
##########################

from functions import *

size = range(len(cars_df))

for ind in size:
    series = cars_df.iloc[ind]
    if 'All-electric' in series['Category']:
        series['Category'] = 'BEV'
    else:
        series['Category'] = 'PHEV'

for ind in size:
    series = cars_df.iloc[ind]
    for word in [' PHEV', ' Plug-in Hybrid', ' Electric']:
        if word in series['Model']:
            series['Model'] = series['Model'].replace(word, '')

cars_df.insert(2, 'Trim', np.zeros(51))
cars_df['Model'] = cars_df.apply(model_formatter, axis=1)
cars_df['Trim'] = cars_df.apply(model_splitter, axis=1)
cars_df['Model'] = cars_df.apply(model_correction, axis=1)

drop_list = []
separators = [' or ', ', ', ' and ']
for ind in size:
    series = cars_df.iloc[ind]
    if any([separator in series['Trim'] for separator in separators]):
            drop_list.append(ind)
            temp= trimseparator_QC(series,' or |, | and ')
            cars_df = cars_df.append(temp, ignore_index=True)
cars_df.drop(drop_list,inplace=True)

drop_list = []
for ind in size:
    series = cars_df.iloc[ind]
    if ' - ' in series['Year-QC']:
        temp= timeseparator_QC(series)
        cars_df = cars_df.append(temp, ignore_index=True)
        drop_list.append(ind)
cars_df.drop(drop_list,inplace=True)



cars_df

,Make,Model,Trim,Rebate-QC,Year-QC,Category
0,Audi,Q4 e-tron,Komfort,$7 000,2022,BEV
5,BMW,i4,eDrive40,$7 000,2022,BEV
6,BYD,E6,,$7 000,2020,BEV
10,Ford,Fusion,,$2 500,2020,PHEV
11,Ford,Mustang Mach-E,Select,$7 000,2022,BEV
...,...,...,...,...,...,...
139,Ford,Escape,SEL,$2 500,2021,PHEV
140,Ford,Escape,SEL,$2 500,2022,PHEV
141,Ford,Escape,Titanium,$2 500,2020,PHEV
142,Ford,Escape,Titanium,$2 500,2021,PHEV


In [25]:
cars_df

,Make,Model,Rebate-QC,Year-QC,Category
0,Audi,Q4 e-tron – Komfort,$7 000,2022,All-electric
1,BMW,X3 – xDrive30e,$2 500,2020 - 2021 - 2022,Plug-in hybrid
2,BMW,3 Series – 330e or 330e xDrive,$2 500,2021 - 2022,Plug-in hybrid
3,BMW,i3,$7 000,2020 - 2021,All-electric
4,BMW,i3 REx and i3s Rex,$5 000,2020 - 2021,Plug-in hybrid
5,BMW,i3s,$7 000,2020 - 2021,All-electric
6,BMW,i4 eDrive40,$7 000,2022,All-electric
7,BYD,E6,$7 000,2020,All-electric
8,Chevrolet,Bolt EUV,$7 000,2022 - 2023,All-electric
9,Chevrolet,Bolt EV,$7 000,2020 - 2021 - 2022 - 2023,All-electric
